<a href="https://colab.research.google.com/github/mhpbreugem/BBP/blob/main/Hellwig_1980.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
using LinearAlgebra, BenchmarkTools, Symbolics, NLsolve

In [187]:
function φSYS(Ψ,ΣF,rf,γ)

    # 00. Extract Variables and Define Random Variables

        a1,a2,b1,b2,c1,c2,d1,d2,ΣΠ = Ψ
        # F = [X, Z, εi]

    # 0. Define Private Information

        S_PRIVATE = [1.0, 0.0, 1.0]     # Signal Y: F.S_PRIVATE

    # 1. Conjecture Price Function (Public Information)

        ## P = a1*Π + a2*Z

        S_PUBLIC = [a1, a2, 0]          # Signal P: F.S_PUBLIC

    # 2. Learning

        S = hcat(S_PRIVATE, S_PUBLIC)

        ΣYY = transpose(S) * ΣF * S                # 2×2
        ΣFY = ΣF * S                               # 3×2
        B = ΣFY * inv(ΣYY)                         # 3×2
        Σ = ΣF - ΣFY * inv(ΣYY) * transpose(ΣFY)   # 3×3

        EQ_LEARN = [b1 - B[1,1], b2 - B[1,2], ΣΠ - Σ[1,1]]

    # 3. Individual Demand (CARA)

        ## Xi = [E[Π|S] - P*(1+rf)] / (γ*ΣΠ)
        ##    = {b1*Y + [b2 - (1+rf)]*P} / (γ*ΣΠ)

        EQ_DEMAND = [c1 - b1/(γ*ΣΠ), c2 - b2*(1+rf)/(γ*ΣΠ)]

    # 4. Demand Aggregation

        ## Due to i.i.d. εi, μi aggregates to Π
        ## Xbar(=0) - Z = d1*Π + d2*P

        EQ_AGGREGATION = [d1 - c1, d2 - c2]

    # 5. Market Clearing

        ## TODO: FOR NOW: Xbar = 0
        ## P = (-d2/d1)*Π + (-1/d2)*Z
        ##   = a1*Π + a2*Z

        EQ_MC = [a1 - (-d2/d1), a2 - (-1/d2)]

    return vcat(EQ_LEARN, EQ_DEMAND, EQ_AGGREGATION,EQ_MC)
    #return a1^2,a2^2,b1^2,b2^2,c1^2,c2^2,d1^2,d2^2,k1^2,k2^2,(Σ-1)^2
end

φSYS (generic function with 4 methods)

In [203]:
φSYS([0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5],diagm([1, 1, 1]),0,1)

9-element Vector{Float64}:
  0.16666666666666669
 -0.16666666666666663
  0.16666666666666663
 -0.5
 -0.5
  0.0
  0.0
  1.5
  2.5

In [207]:
x0 = 0.5*ones(9)  # Initial guess for the 9 parameters
sol = nlsolve(x -> φSYS(x, diagm([1, 1, 1]),0.01,1), x0)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
 * Zero: [-0.00412725546985542, -242.2887029292894, 0.5000000980929187, 0.0013711330696727427, 1.0000080321330063, 0.0034485042346326295, 1.0000080482478226, 0.0041273040340829665, 0.4999961061013369]
 * Inf-norm of residuals: 0.001371
 * Iterations: 1000
 * Convergence: false
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: false
 * Function Calls (f): 1000
 * Jacobian Calls (df/dx): 985